In [ ]:
!pip install -q finta
!pip install -q pycaret shap
!pip install -q scikit-learn==0.23.2

     |████████████████████████████████| 266kB 4.0MB/s 
     |████████████████████████████████| 358kB 20.6MB/s 
     |████████████████████████████████| 6.8MB 25.4MB/s 
     |████████████████████████████████| 256kB 39.7MB/s 
     |████████████████████████████████| 1.7MB 44.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174kB 43.1MB/s 
     |████████████████████████████████| 2.0MB 44.2MB/s 
     |████████████████████████████████| 276kB 49.7MB/s 
     |████████████████████████████████| 112kB 56.0MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 14.2MB 188kB/s 
     |████████████████████████████████| 1.4MB 47.9MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 3.1MB 41.5MB/s

In [ ]:
# from sklearn.tree import DecisionTreeClassifier 
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.model_selection import train_test_split
# from sklearn import metrics 

In [ ]:
from pycaret.classification import *

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
from finta import TA

In [ ]:
import pickle

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/programming/full_data.csv')

In [ ]:
df = df[['Close' , 'Open' , 'High' , 'Low']]

In [ ]:
df['21MA'] = TA.EMA(df,21)
df['9MA'] = TA.EMA(df,9)

In [ ]:
df = df[['Close' , '21MA' , '9MA']]
df.head()

,Close,21MA,9MA
0,1520.799,1520.799000,1520.799000
1,1520.986,1520.896952,1520.902889
2,1519.842,1520.511305,1520.468098
3,1519.984,1520.360078,1520.304108
4,1519.618,1520.182116,1520.100007


In [ ]:
X_next = 15

In [ ]:
prev_9ma = df['9MA'].shift(1)
prev_21ma = df['21MA'].shift(1)

crossing = (((df['9MA'] <= df['21MA']) & (prev_9ma >= prev_21ma))
            | ((df['9MA'] >= df['21MA']) & (prev_9ma <= prev_21ma)))

df['crossover'] = crossing
df['crossover_prediction'] = df['crossover'].shift(-X_next)
df

,Close,21MA,9MA,crossover,crossover_prediction
0,1520.799,1520.799000,1520.799000,False,False
1,1520.986,1520.896952,1520.902889,True,False
2,1519.842,1520.511305,1520.468098,True,False
3,1519.984,1520.360078,1520.304108,False,False
4,1519.618,1520.182116,1520.100007,False,False
...,...,...,...,...,...
105088,1778.505,1778.090970,1778.220162,False,NaN
105089,1778.186,1778.099609,1778.213330,False,NaN
105090,1778.215,1778.110099,1778.213664,False,NaN
105091,1778.705,1778.164181,1778.311931,False,NaN


In [ ]:
df['9MA'] = df['9MA'].pct_change()
df['21MA'] = df['21MA'].pct_change()

In [ ]:
df = df.fillna(0)

In [ ]:
df = df[['9MA' , '21MA' , 'crossover_prediction']]
df['crossover_prediction'] = df['crossover_prediction'].astype('int')

In [ ]:
df = df.iloc[:-X_next]

In [ ]:
y = df.loc[ : , 'crossover_prediction'].values

In [ ]:
y_min = len(y[y == 1])

In [ ]:
indexes = np.where(y == 0)[0][y_min:]
df = df.drop(df.index[indexes])

In [ ]:
# X = df.loc[ :df.shape[0] - X_next , ['9MA' , '21MA']].values
# y = df.loc[ :df.shape[0] - X_next , 'crossover_prediction'].values

In [ ]:
# y = y.astype('int')

In [ ]:
# y_min = len(y[y == 1])

In [ ]:
# indexes = np.where(y == 0)[0][y_min:]
# y = np.delete(y , indexes)
# X = np.delete(X , indexes , axis=0)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# clf = LinearDiscriminantAnalysis()

# clf = clf.fit(X_train,y_train)

# y_pred = clf.predict(X_test)

In [ ]:
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5082187185508219


In [ ]:
# filename = 'FOREX_MA.sav'
# pickle.dump(clf, open(filename, 'wb'))

In [ ]:
exp_clf = setup(data = df, target = 'crossover_prediction')

,Description,Value
0,session_id,8355
1,Target,crossover_prediction
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(9934, 3)"
5,Missing Values,False
6,Numeric Features,2
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.5327,0.5240,0.3602,0.5494,0.4346,0.0651,0.0695,0.207
qda,Quadratic Discriminant Analysis,0.5293,0.5362,0.2465,0.5658,0.3420,0.0580,0.0701,0.018
gbc,Gradient Boosting Classifier,0.5245,0.5312,0.3956,0.5331,0.4537,0.0488,0.0508,0.430
lightgbm,Light Gradient Boosting Machine,0.5168,0.5216,0.4641,0.5180,0.4894,0.0334,0.0336,0.139
knn,K Neighbors Classifier,0.5150,0.5204,0.5062,0.5150,0.5104,0.0301,0.0301,0.132
lda,Linear Discriminant Analysis,0.5101,0.5056,0.4869,0.5104,0.4981,0.0202,0.0203,0.020
et,Extra Trees Classifier,0.5081,0.5093,0.5027,0.5075,0.5050,0.0162,0.0162,0.814
rf,Random Forest Classifier,0.5040,0.5077,0.5007,0.5032,0.5019,0.0079,0.0079,1.230
nb,Naive Bayes,0.5016,0.5004,0.5607,0.5032,0.5207,0.0035,0.0036,0.019
lr,Logistic Regression,0.5005,0.5084,0.0000,0.0000,0.0000,0.0000,0.0000,0.312


In [ ]:
tuned_model = tune_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5359,0.5240,0.3736,0.5532,0.4460,0.0718,0.0760
1,0.5057,0.5092,0.3276,0.5089,0.3986,0.0115,0.0123
2,0.5661,0.5685,0.3966,0.6000,0.4775,0.1322,0.1405
3,0.5266,0.5320,0.3256,0.5433,0.4072,0.0527,0.0575
4,0.5468,0.5542,0.3977,0.5656,0.4670,0.0931,0.0975
5,0.5540,0.5578,0.3919,0.5787,0.4674,0.1075,0.1136
6,0.5712,0.5899,0.3775,0.6150,0.4679,0.1420,0.1539
7,0.5151,0.5182,0.3746,0.5200,0.4355,0.0298,0.0311
8,0.5295,0.5323,0.3890,0.5400,0.4523,0.0586,0.0610
9,0.5381,0.5341,0.3689,0.5565,0.4437,0.0758,0.0805


In [ ]:
save_model(tuned_model , model_name='FOREX_MA')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='crossover_prediction',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None...
                 ('dummy', Dummify(target='crossover_prediction')),
                 ('fix_perfect', Remove_100(target='crossover_prediction')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'pa